# Text Analysis
Analyzes texts from seemingly-similar authors, Rachel Swirsky, Cat Valente, and Carmen Maria Machado, on a sentence-by-sentence basis, to see if their writing styles can be predicted.

In [1]:
import numpy as np
import pandas as pd
import spacy
import re
from urllib.request import Request, urlopen
import requests
from bs4 import BeautifulSoup
from time import time
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', 100)
%matplotlib inline

## Scraping Data

### Requesting and parsing html
* Using requests and BeautifulSoup
* The program first pulls stories directly from their published websites

In [2]:
# Pull stories from their websites

header={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
        AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36'}

def get_raw(url, no_strong=False, no_b=False):
    response = requests.get(url, headers=header)
#     req = Request(url, headers=header)
    html = response.content
#     html = urlopen(req).read().decode('utf8')
    soup = BeautifulSoup(html, 'lxml')
    if no_strong == True:
        for strong in soup('strong'):
            strong.decompose()
    if no_b == True:
        for b in soup('b'):
            b.decompose()
    return soup.get_text()

t0 = time()

# Swirsky stories
dino_raw = get_raw('https://www.apex-magazine.com/if-you-were-a-dinosaur-my-love/')

eros_raw = get_raw('https://www.tor.com/2009/03/03/eros-philia-agape/')

still_raw = get_raw('http://uncannymagazine.com/article/love-is-never-still/', no_b=True)

# Valente stories
fade_raw = get_raw('http://clarkesworldmagazine.com/valente_08_12/')

silent1_raw = get_raw('http://clarkesworldmagazine.com/valente_10_11/', no_strong=True)

silent2_raw = get_raw('http://clarkesworldmagazine.com/valente_11_11/', no_strong=True)

silent3_raw = get_raw('http://clarkesworldmagazine.com/valente_12_11/', no_strong=True)

lion_raw = get_raw('http://uncannymagazine.com/article/planet-lion/')

# Machado stories
stitch_raw = get_raw('https://granta.com/The-Husband-Stitch/')

follow_raw = get_raw('http://www.lightspeedmagazine.com/fiction/help-follow-sister-land-dead/')

descent_raw = get_raw('http://www.nightmare-magazine.com/fiction/descent/')

bites_raw = get_raw('http://texas.gulfcoastmag.org/journal/29.2-summer/fall-2017/eight-bites/')

print('Get raw in {:.2f} seconds'.format(time() - t0))

Get raw in 11.18 seconds


### Finding story text in html
* Using find() from BeautifulSoup
* Identifies story text inside the total site text

In [3]:
# Select out story text

def select_out(raw, start_text, end_text):
    start = raw.find(start_text)
    end = raw.find(end_text)
    return raw[start:end+len(end_text)]

dinosaur = select_out(dino_raw, 
                      'If you were a dinosaur, my', 
                      ', and the stuttering of my broken heart.'
                     )
eros = select_out(eros_raw, 
                  'Lucian packed his possessions before he left.', 
                  'yet form the thoughts to wonder what will happen next.\nHe moves on.'
                 )
still = select_out(still_raw, 
                  'Through every moment of carving, I want her as one wants a woman. I want', 
                  'and decay, but it is also the early bloom that opens into winter’s cold. She is love, and she will always be reborn.'
                  )
fade = select_out(fade_raw, 
                 'ZOOM IN on a bright-eyed Betty in a crisp green dress,', 
                 'And over the black, a cheerful fat man giving the thumbs up to Sylvie, grinning:\n\nBuy Freedom Brand Film! It\'s A-OK!'
                 )
silent1 = select_out(silent1_raw, 
                    'Inanna was called Queen of Heaven and Earth,', 
                    'The sun breaks the mountain crests, hard and cold, a shaft of white spilling over the black lake.'
                    )
silent2 = select_out(silent2_raw, 
                    'Humanity lived many years and ruled the earth,', 
                    'The castle windows go dark, one by one.'
                    )
silent3 = select_out(silent3_raw, 
                    'Tell me a story about yourself, Elefsis.', 
                    'we walk up the long path out of the churning, honey-colored sea.'
                    )
silent = silent1 + '\n' + silent2 + '\n' + silent3

lion = select_out(lion_raw, 
                 'Initial Survey Report: Planet 6MQ441 (Bakeneko), Alaraph', 
                 'Szent Istvan. She longs to hear the first roar of her young.'
                 )
stitch = select_out(stitch_raw, 
                   '(If you read this story out loud, please use', 
                   'backwards off my neck and rolls off the bed, I feel as lonely as I have ever been.'
                   )
follow = select_out(follow_raw, 
                   'Help Me Follow My Sister\ninto the Land of the Dead\nby Ursula Ruiz', 
                   's—in a terrifying place where no mortal has any business treading.'
                   ) + '\n' + select_out(follow_raw, 
                                        'Home\nThis is the thing about my sister and I:', 
                                        'Lucille L\nJun 28, 2015\nUrsula?'
                                        )
descent = select_out(descent_raw, 
                    'We gathered for the last time in October,', 
                    ', drinks cocked in their hands.\nI looked down.'
                    )
bites = select_out(bites_raw, 
                  'As they put me to sleep, my mouth fills with the dust of the moon. I expect to choke on the silt but', 
                  'my charge.\"I\'m sorry,\" I will whisper into her as she walks me toward the front door.\"I\'m sorry,\" I will repeat. \"I didn\'t know.\"')

## Parsing Text

### Cleaning 
* Cleaning out un-parseable symbols
* Using re

In [4]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub(r'[\[\]]', '', text)
    text = ' '.join(text.split())
    return text

In [5]:
# Clean out bad symbols
dinosaur = text_cleaner(dinosaur)
eros = text_cleaner(eros)
fade = text_cleaner(fade)
silent = text_cleaner(silent)
lion = text_cleaner(lion)
still = text_cleaner(still)
stitch = text_cleaner(stitch)
follow = text_cleaner(follow)
descent = text_cleaner(descent)
bites = text_cleaner(bites)

### Parsing documents
* Using spaCy
* Identifies sentence structure in the stories

In [6]:
t0 = time()

# Parse using spaCy
nlp = spacy.load('en')

dinosaur_doc = nlp(dinosaur)
eros_doc = nlp(eros)
fade_doc = nlp(fade)
silent_doc = nlp(silent)
lion_doc = nlp(lion)
still_doc = nlp(still)
stitch_doc = nlp(stitch)
follow_doc = nlp(follow)
descent_doc = nlp(descent)
bites_doc = nlp(bites)

print('Parsed in {:.2f} seconds'.format(time() - t0))

Parsed in 24.07 seconds


### Grouping data into sentences
* We use spaCy's tokenization and sentence identification
* Creates one table of all the data, organized with one sentence for each line

In [7]:
# Group sentences with their authors. (Each sentence will be a row.)

dinosaur_sents = [["Swirsky", sent.string] for sent in dinosaur_doc.sents]
eros_sents = [["Swirsky", sent.string] for sent in eros_doc.sents]
still_sents = [["Swirsky", sent.string] for sent in still_doc.sents]
fade_sents = [["Valente", sent.string] for sent in fade_doc.sents]
silent_sents = [["Valente", sent.string] for sent in silent_doc.sents]
lion_sents = [["Valente", sent.string] for sent in lion_doc.sents]
stitch_sents = [["Machado", sent.string] for sent in stitch_doc.sents]
follow_sents = [["Machado", sent.string] for sent in follow_doc.sents]
descent_sents = [["Machado", sent.string] for sent in descent_doc.sents]
bites_sents = [["Machado", sent.string] for sent in bites_doc.sents]

In [8]:
# Take part of speech of each word in each sentence.

dinosaur_pos = [[token.pos_ for token in sent] for sent in dinosaur_doc.sents]
eros_pos = [[token.pos_ for token in sent] for sent in eros_doc.sents]
still_pos = [[token.pos_ for token in sent] for sent in still_doc.sents]
fade_pos = [[token.pos_ for token in sent] for sent in fade_doc.sents]
silent_pos = [[token.pos_ for token in sent] for sent in silent_doc.sents]
lion_pos = [[token.pos_ for token in sent] for sent in lion_doc.sents]
stitch_pos = [[token.pos_ for token in sent] for sent in stitch_doc.sents]
follow_pos = [[token.pos_ for token in sent] for sent in follow_doc.sents]
descent_pos = [[token.pos_ for token in sent] for sent in descent_doc.sents]
bites_pos = [[token.pos_ for token in sent] for sent in bites_doc.sents]

In [9]:
# list(dinosaur_doc.print_tree())

In [10]:
# Combine the sentences from the stories into one data frame.
sentences = pd.DataFrame(dinosaur_sents + eros_sents + still_sents + fade_sents + silent_sents + lion_sents + stitch_sents + follow_sents + descent_sents + bites_sents)
sentences.columns = ['author', 'text']

# Strip out unnecessary punctuation.
sentences['text'] = sentences['text'].str.lower().str.replace(r'[^a-zA-Z0-9 ]+', ' ').fillna('')

# Word count per sentence.
sentences['words'] = sentences['text'].str.split().apply(len)

# Parts of speech classification of each word in sentence.
sentences['pos'] = pd.Series(dinosaur_pos + eros_pos + still_pos + fade_pos + silent_pos + lion_pos + stitch_pos + follow_pos + descent_pos + bites_pos)

# Remove blank sentences.
sentences = sentences.loc[sentences['words'] > 0]

# Remove proper nouns personal to each document.
propn_pat = r'\b(agogna|koetoi|adriana|lucian|galatea|aphrodite|hephaestus|fuoco|inanna|enki|erishkigal|salma|janet|lawrence|olive|luna|ursula|ceno|neva|elefsis|ravan|cassian|diane|ares|ben|seki|ilet)\b'
sentences['text'] = sentences['text'].str.replace(propn_pat, '')

# Sentence counts by author.
df_count = sentences.author.value_counts().to_frame(name='count')
df_count['ratio'] = (df_count['count'] / len(sentences.author)).round(2)
print('Full Data Set:')
display(df_count)
print('{} total sentences'.format(len(sentences.author)))
display(sentences.loc[sentences['author'] == 'Swirsky'].head(1))
display(sentences.loc[sentences['author'] == 'Valente'].head(1))
display(sentences.loc[sentences['author'] == 'Machado'].head(1))

Full Data Set:


,count,ratio
Valente,2269,0.39
Swirsky,1880,0.33
Machado,1603,0.28


5752 total sentences


,author,text,words,pos
0,Swirsky,if you were a dinosaur my love then you woul...,14,"[ADP, PRON, VERB, DET, NOUN, PUNCT, ADJ, NOUN,..."


,author,text,words,pos
2153,Valente,was called queen of heaven and earth queen o...,37,"[PROPN, VERB, VERB, PROPN, ADP, PROPN, CCONJ, ..."


,author,text,words,pos
4486,Machado,if you read this story out loud please use t...,24,"[PUNCT, ADP, PRON, VERB, DET, NOUN, ADV, ADV, ..."


## Splitting Test Set
This protects the vectorizer and regressor from over-fitting to the training set.

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(sentences.drop(['author', 'words'], axis=1), sentences['author'], test_size=0.2)

In [32]:
print('Training Set:')
print(y_train.value_counts().to_frame(name='count'))
print('\nTest Set:')
print(y_test.value_counts().to_frame(name='count'))

Training Set:
         count
Valente   1823
Swirsky   1525
Machado   1253

Test Set:
         count
Valente    446
Swirsky    355
Machado    350


## Pipeline: Selector, Transformers and Estimator

### Item Selector
* Makes it possible to run different transformers on different columns of data before concatenating them together with FeatureUnion and then running through the estimator

In [33]:
from sklearn.base import BaseEstimator, TransformerMixin
# The key corresponding to the desired value in a mappable.

class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]

### TFIDF Vectorizer

* Try most common words across all texts
* Bigrams and trigrams are arrangements of two- and three-words, respectively

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(analyzer=str.split, 
                            stop_words='english', 
                            lowercase=True, 
                            use_idf=True, 
                            norm='l2', 
                            smooth_idf=True, 
                            max_features=None, 
                            ngram_range=(1,3), 
                            max_df=0.6, 
                            min_df=1)

### Text Statistics

In [35]:
class TextStats(BaseEstimator, TransformerMixin):
# Extract features from each document for DictVectorizer

    def fit(self, x, y=None):
        return self

    def transform(self, sents):
        return [{'length': len(sent), 
                 'ADJ': sent.count('ADJ'), 
                 'NOUN': sent.count('NOUN'), 
                 'VERB': sent.count('VERB'), 
                 'ADV': sent.count('ADV'), 
                 'ADP': sent.count('ADP'), 
                 'CCONJ': sent.count('CCONJ'), 
                 'INTJ': sent.count('INTJ'), 
                 'PART': sent.count('PART'), 
                 'SCONJ': sent.count('SCONJ'), 
                 'PRON': sent.count('PRON'), 
                 'PROPN': sent.count('PROPN'), 
                 'DET': sent.count('DET'), 
                 'NUM': sent.count('NUM'), 
                 'AUX': sent.count('AUX'), 
                 'SYM': sent.count('SYM'), 
                }
                for sent in sents]

### Dict Vectorizer

In [36]:
from sklearn.feature_extraction import DictVectorizer
# list of dicts -> feature matrix

### Stochastic Gradient Descent Classifier
* Combines support vector classifier with gradient descent

In [37]:
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier(penalty='elasticnet', 
                    alpha=0.0001, 
                    class_weight=None, 
                    loss='log', 
                    tol=0.005, 
                    fit_intercept=True, 
                    power_t=0.5)

### Pipeline
* Chains transformers and estimator as input to grid search algorithm

In [38]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

pipe = Pipeline([
    ('union', FeatureUnion(
        transformer_list=[
            ('text', Pipeline([
                ('selector', ItemSelector(key='text')), 
                ('vect', vectorizer), 
            ])), 
            ('pos', Pipeline([
                ('selector', ItemSelector(key='pos')), 
                ('stats', TextStats()), 
                ('dict', DictVectorizer()), 
            ])), 
        ], 
        transformer_weights={
            'text': 1.0, 
            'pos': 1.0, 
        }, 
    )), 
    ('model', sgd), 
])

## Grid Search
### Hyperparameters
* List parameters to search over for best cv fit

In [39]:
vect_features = [None, 20000, 10000, 5000, 1000]
vect_ngrams = [(1,1), (1,2), (1,3)]
vect_max = [0.6, 0.7, 0.8]
vect_min = [1, 2, 3]
vect_norm = ['l2', 'l1', None]
model_alpha = [0.001, 0.0001, 0.00001]
model_loss = ['hinge', 'log', 'squared_hinge', 'perceptron']
model_tol = [0.01, 0.005, 0.001]
model_class_weight = ['balanced', None]
model_power_t = [0.4, 0.5, 0.6]
model_penalty = ['l2', 'elasticnet', 'l1', 'none']
model_intercept = [True, False]

parameters = {
#         'union__text__vect__max_features': vect_features, 
        'union__text__vect__ngram_range': vect_ngrams, 
#         'union__text__vect__max_df': vect_max, 
#         'union__text__vect__min_df': vect_min, 
#         'union__text__vect__norm': vect_norm, 
        'model__alpha': model_alpha, 
#         'model__loss': model_loss, 
#         'model__tol': model_tol,
#         'model__class_weight': model_class_weight, 
#         'model__power_t': model_power_t, 
#         'model__penalty': model_penalty, 
#         'model__fit_intercept': model_intercept
    }

from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(pipe, 
                    parameters, 
                    scoring='f1_weighted', 
                    return_train_score=True)

### Fitting and Results

In [40]:
print('Performing grid search...')
t0 = time()
grid.fit(X_train, y_train)
print('\nCompleted in {:.2f} seconds'.format(time() - t0))

Performing grid search...

Completed in 11.52 seconds


In [66]:
print('\nBest Training Set Score: {:.2%}'.format(grid.cv_results_['mean_train_score'][grid.best_index_]))
print('\nBest Cross-Val Score: {:.2%}'.format(grid.best_score_))

print('\nBest Parameters:\n')
best_parameters = grid.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print('{}: {}'.format(param_name, best_parameters[param_name]))

from sklearn.metrics import confusion_matrix

test_score = grid.score(X_test, y_test)
y_test_pred = grid.predict(X_test)
test_cm = confusion_matrix(y_test, y_test_pred)

print('\nTest set score: {:.1%}'.format(test_score))

df_test_cm = pd.DataFrame(test_cm, index=grid.classes_, columns=grid.classes_)
df_test_cm.index.name = 'Actual Author'
df_test_cm.columns.name = 'Predicted Author'
display(df_test_cm)

feature_list1 = grid.best_estimator_.named_steps['union'].transformer_list[0][1].named_steps['vect'].get_feature_names()
feature_list2 = grid.best_estimator_.named_steps['union'].transformer_list[1][1].named_steps['dict'].get_feature_names()
feature_list = feature_list1 + feature_list2
feature_coefficients = pd.DataFrame(grid.best_estimator_.steps[-1][1].coef_, columns=feature_list, index=grid.classes_)
print('\n{} features'.format(len(feature_list)))
# columns=feature_list, 
display(feature_coefficients.sort_values('Machado', ascending=False, axis=1).head(10))
display(feature_coefficients.sort_values('Swirsky', ascending=False, axis=1).head(10))
display(feature_coefficients.sort_values('Valente', ascending=False, axis=1).head(10))


Best Training Set Score: 48.27%

Best Cross-Val Score: 43.71%

Best Parameters:

model__alpha: 0.0001
union__text__vect__ngram_range: (1, 2)

Test set score: 48.7%


Predicted Author,Machado,Swirsky,Valente
Actual Author,,,
Machado,90,226,34
Swirsky,12,320,23
Valente,32,238,176



7836 features


,my,says,SYM,said,ribbon,woman,son,going,i,husband,cal,2015,s,sister,jun,then,who,girl,dead,after,class,our,hand,back,around,about,next,school,here,27,surgery,walk,doctor,l,lucille,nurse,story,off,asks,tell,door,shooting,from,open,there,down,through,just,students,land,...,hundred,hope,robot,smallgod,take,made,both,too,now,one,stand,bird,only,do,hole,watering,thought,blue,make,inside,yet,her,their,sea,moment,all,most,every,human,would,big,in,called,must,loved,knew,lions,of,never,rose,cannot,could,how,them,d,love,lion,PROPN,as,not
Machado,22.669156,11.842142,11.048558,10.401166,10.048437,8.773188,8.559374,8.049587,7.984426,7.883196,7.665648,7.643355,7.545662,7.519693,7.424308,7.179061,7.087719,6.726148,6.473500,6.374264,6.314750,6.310563,6.067807,5.940608,5.891107,5.824749,5.698153,5.672158,5.627450,5.487280,5.452226,5.409450,5.208804,5.147824,5.147824,5.045584,4.982442,4.912084,4.891023,4.875701,4.858429,4.857788,4.722126,4.588280,4.556009,4.523513,4.477635,4.469875,4.464070,4.410702,...,-3.449303,-3.454620,-3.467994,-3.540555,-3.575574,-3.645247,-3.679453,-3.747942,-3.783498,-3.800957,-3.848133,-3.879291,-3.957983,-3.986645,-4.072615,-4.072615,-4.112084,-4.321137,-4.431504,-4.432290,-4.433114,-4.478945,-4.566977,-4.644705,-4.658308,-4.680072,-4.706962,-4.715632,-4.907267,-5.273314,-5.342299,-5.404815,-5.436521,-5.500072,-5.506426,-5.735379,-5.994114,-6.165773,-6.229015,-6.706473,-6.742471,-6.777801,-7.077931,-7.140369,-7.148018,-7.596391,-8.050014,-8.179629,-8.870049,-11.288590
Swirsky,-6.635164,-12.166512,-2.297937,-0.634507,-5.779481,0.498848,-4.672695,-3.200692,-15.878871,-0.983138,-2.974154,-0.547706,2.785731,-4.849636,-0.097513,2.626543,-4.869771,-4.301930,-5.048159,-4.382985,-2.161292,-6.172538,0.339117,-3.493112,-0.953915,-10.227194,-1.858022,-2.161456,-1.730077,-0.042241,-3.733291,-1.784442,-2.268810,-0.244032,-0.244032,-2.343474,-6.104334,-4.020412,-4.066215,-4.524015,0.000000,-0.956415,5.286348,-2.495448,-2.587108,0.000000,-0.480501,-5.996460,-1.713990,-1.653926,...,0.000000,2.363834,4.315235,-4.762257,6.968103,0.000000,4.881134,4.337320,4.791251,-2.955717,1.959192,6.984470,2.030533,1.524441,-5.132439,-5.132439,0.065300,-2.968782,-0.765931,-4.934073,-0.840040,3.516437,-2.284479,-2.173783,7.168468,-1.595983,-0.837171,-0.435254,0.000000,2.122041,-3.955127,-4.717167,-5.388159,6.342876,5.026831,0.000000,-5.503268,-3.724726,1.014182,23.112758,2.094389,-0.911165,-1.013355,2.656178,16.099560,16.095870,-10.089249,0.000000,6.381459,-3.831908
Valente,-10.897042,0.000000,-5.766490,-5.225661,-4.283806,-8.302325,-4.288854,-4.248840,4.052455,-5.484382,-3.442419,-6.429341,-11.370221,-2.688588,-6.464852,-9.879814,-1.063265,-2.949501,-1.141677,-1.632489,-3.622262,-0.523153,-7.066983,-1.403037,-3.836568,4.461239,-3.402336,-2.699639,-3.563248,-4.754880,-1.030051,-2.373085,-2.836523,-4.071546,-4.071546,-2.272412,1.381569,-0.152908,-2.120047,-1.439275,-2.984368,-3.248747,-7.994920,-2.158393,-2.151739,-3.372501,-3.646525,0.630020,-2.221086,-2.924630,...,3.043981,0.372509,-0.816174,6.896395,-2.158295,4.410902,-0.397079,0.000000,0.000000,7.157446,1.208787,-3.382943,1.777967,0.699306,8.318252,8.318252,4.581147,6.384492,4.328675,8.130285,4.605484,0.000000,4.425630,5.559520,-2.678761,5.171155,5.132662,3.518713,3.675108,1.716638,8.121333,7.994719,10.513214,-1.815175,0.072290,5.339060,10.873478,8.618723,4.315874,-16.133043,3.291316,6.318045,6.929031,3.973968,-9.891404,-8.638722,17.563369,2.737807,2.820095,11.955432


,rose,he,his,d,love,him,will,the,ivory,across,moment,bird,take,as,must,toward,she,okay,sun,against,movement,from,roses,gods,another,loved,both,until,now,mortal,driver,too,robot,course,felt,that,mine,pedestal,things,cold,between,hammer,sculptor,fingers,sand,bronze,statue,beauty,wine,teenager,...,after,wanted,interior,tell,anything,long,be,me,looks,son,you,in,dreaming,smallgod,show,sister,myself,who,know,inside,out,everyone,dead,some,hole,watering,up,called,lions,ribbon,just,story,our,come,because,very,NUM,my,is,had,say,did,like,mother,it,lion,about,we,says,i
Machado,-6.706473,-2.617046,-1.562500,-7.148018,-7.596391,-1.647750,2.054882,0.000000,-3.192381,-0.626256,-4.658308,-3.879291,-3.575574,-8.870049,-5.500072,-1.299605,-0.502178,-2.571710,-3.208045,0.000000,-1.723107,4.722126,-1.163003,-1.689779,0.000000,-5.506426,-3.679453,0.000000,-3.783498,-1.613840,-1.319377,-3.747942,-3.467994,-3.089173,-0.862789,0.834072,-2.396914,-2.046487,-3.177976,1.272850,0.000000,-1.356810,-2.424577,0.000000,-2.483793,-1.396814,-1.426098,-1.097238,-0.090757,-0.431727,...,6.374264,-1.262734,-2.444426,4.875701,-0.012591,-2.017722,-0.796409,0.000000,2.841041,8.559374,-2.370411,-5.404815,0.000000,-3.540555,-2.511755,7.519693,-2.472921,7.087719,0.736805,-4.432290,3.952581,2.351971,6.473500,-1.092405,-4.072615,-4.072615,3.723510,-5.436521,-5.994114,10.048437,4.469875,4.982442,6.310563,4.066972,3.205426,1.923103,1.193460,22.669156,2.676463,-1.060842,3.788049,-2.148434,2.222717,0.886160,-2.852432,-8.050014,5.824749,-2.226220,11.842142,7.984426
Swirsky,23.112758,16.959250,16.324014,16.099560,16.095870,15.057087,8.867636,7.372473,7.352147,7.202342,7.168468,6.984470,6.968103,6.381459,6.342876,5.870017,5.545917,5.525992,5.397259,5.347018,5.299621,5.286348,5.232887,5.136532,5.032944,5.026831,4.881134,4.804121,4.791251,4.626975,4.583931,4.337320,4.315235,4.293227,4.289628,4.269382,4.249319,4.236529,4.108501,4.004300,3.923776,3.885088,3.874079,3.866180,3.721641,3.716974,3.710350,3.668829,3.659829,3.536218,...,-4.382985,-4.388791,-4.439469,-4.524015,-4.564370,-4.576305,-4.615609,-4.619610,-4.637326,-4.672695,-4.686129,-4.717167,-4.735424,-4.762257,-4.812787,-4.849636,-4.853427,-4.869771,-4.903907,-4.934073,-4.956084,-5.011280,-5.048159,-5.119984,-5.132439,-5.132439,-5.346318,-5.388159,-5.503268,-5.779481,-5.996460,-6.104334,-6.172538,-6.333430,-6.455775,-6.472115,-6.623971,-6.635164,-6.823767,-7.143609,-7.532305,-7.849918,-8.116790,-9.290286,-9.442536,-10.089249,-10.227194,-10.923508,-12.166512,-15.878871
Valente,-16.133043,-14.337742,-12.029855,-9.891404,-8.638722,-12.417732,-9.612672,-5.028709,-3.465229,-4.262057,-2.678761,-3.382943,-2.158295,2.820095,-1.815175,-3.220434,-3.606992,-2.618766,-2.434592,-5.745933,-3.167140,-7.994920,-2.948461,-3.539772,-4.434956,0.072290,-0.397079,-3.932510,0.000000,-2.589746,-2.539940,0.000000,-0.816174,-0.944797,-3.293495,-5.435205,-1.338505,-1.803806,-0.949058,-4.069691,-3.616961,-2.377980,-1.792522,-3.195646,-1.666927,-1.446473,-2.012162,-2.149598,-2.706922,-2.548795,...,-1.632489,4.729740,5.573781,-1.439275,4.814779,5.390259,3.646985,4.126403,0.272231,-4.288854,6.151358,7.994719,3.759049,6.896395,7.318667,-2.688588,6.648363,-1.063265,2.606265,8.130285,0.000000,2.178751,-1.141677,4.438479,8.318252,8.318252,1.977393,10.513214,10.873478,-4.283806,0.630020,1.381569,-0.523153,2.972905,2.792239,3.851192,3.643358,-10.897042,3.481985,7.942432,1.002436,7.368961,4.947805,7.286787,10.600957,17.563369,4.461239,10.064493,0.000000,4.052455


,lion,not,lions,it,called,we,of,hole,watering,inside,big,in,had,did,show,mother,one,how,smallgod,dormouse,myself,steelveldt,blue,could,you,well,black,child,dreambody,interior,sea,yttrium,long,knew,jewel,done,all,learn,most,us,like,sludge,uplink,body,anything,experience,they,wanted,machine,yet,...,next,cal,ivory,gods,here,she,between,class,through,around,until,when,cold,lucille,l,beneath,why,feet,going,across,ribbon,son,father,another,27,see,away,the,said,but,that,husband,against,SYM,were,2015,jun,hand,from,woman,love,will,then,d,my,s,his,him,he,rose
Machado,-8.050014,-11.288590,-5.994114,-2.852432,-5.436521,-2.226220,-6.165773,-4.072615,-4.072615,-4.432290,-5.342299,-5.404815,-1.060842,-2.148434,-2.511755,0.886160,-3.800957,-7.077931,-3.540555,-2.874805,-2.472921,-2.493015,-4.321137,-6.777801,-2.370411,-2.154106,-1.638016,-1.604124,-2.526227,-2.444426,-4.644705,-2.054582,-2.017722,-5.735379,-2.436759,-1.401517,-4.680072,-1.211883,-4.706962,-3.054908,2.222717,-1.630347,-2.646500,-2.199484,-0.012591,-2.163291,-2.414287,-1.262734,-2.516446,-4.433114,...,5.698153,7.665648,-3.192381,-1.689779,5.627450,-0.502178,0.000000,6.314750,4.477635,5.891107,0.000000,1.319438,1.272850,5.147824,5.147824,2.143022,3.367415,0.723389,8.049587,-0.626256,10.048437,8.559374,0.000000,0.000000,5.487280,4.041428,4.368790,0.000000,10.401166,3.046534,0.834072,7.883196,0.000000,11.048558,4.234344,7.643355,7.424308,6.067807,4.722126,8.773188,-7.596391,2.054882,7.179061,-7.148018,22.669156,7.545662,-1.562500,-1.647750,-2.617046,-6.706473
Swirsky,-10.089249,-3.831908,-5.503268,-9.442536,-5.388159,-10.923508,-3.724726,-5.132439,-5.132439,-4.934073,-3.955127,-4.717167,-7.143609,-7.849918,-4.812787,-9.290286,-2.955717,-1.013355,-4.762257,-3.404620,-4.853427,-3.649331,-2.968782,-0.911165,-4.686129,-4.098845,-4.339915,-4.012347,-3.308662,-4.439469,-2.173783,-3.354572,-4.576305,0.000000,-3.446423,-3.547670,-1.595983,-4.108559,-0.837171,-2.126761,-8.116790,-3.901082,-2.760753,-2.751338,-4.564370,-2.727745,-2.883569,-4.388791,-2.369459,-0.840040,...,-1.858022,-2.974154,7.352147,5.136532,-1.730077,5.545917,3.923776,-2.161292,-0.480501,-0.953915,4.804121,3.525075,4.004300,-0.244032,-0.244032,1.573374,-0.583617,3.370104,-3.200692,7.202342,-5.779481,-4.672695,2.606212,5.032944,-0.042241,0.000000,2.359604,7.372473,-0.634507,1.496223,4.269382,-0.983138,5.347018,-2.297937,3.196899,-0.547706,-0.097513,0.339117,5.286348,0.498848,16.095870,8.867636,2.626543,16.099560,-6.635164,2.785731,16.324014,15.057087,16.959250,23.112758
Valente,17.563369,11.955432,10.873478,10.600957,10.513214,10.064493,8.618723,8.318252,8.318252,8.130285,8.121333,7.994719,7.942432,7.368961,7.318667,7.286787,7.157446,6.929031,6.896395,6.652816,6.648363,6.452446,6.384492,6.318045,6.151358,6.055679,5.898544,5.724771,5.684131,5.573781,5.559520,5.487857,5.390259,5.339060,5.308985,5.262707,5.171155,5.135332,5.132662,5.074150,4.947805,4.900178,4.897630,4.818724,4.814779,4.768454,4.759480,4.729740,4.676204,4.605484,...,-3.402336,-3.442419,-3.465229,-3.539772,-3.563248,-3.606992,-3.616961,-3.622262,-3.646525,-3.836568,-3.932510,-3.966385,-4.069691,-4.071546,-4.071546,-4.147476,-4.192248,-4.226617,-4.248840,-4.262057,-4.283806,-4.288854,-4.379541,-4.434956,-4.754880,-4.820981,-4.823982,-5.028709,-5.225661,-5.419605,-5.435205,-5.484382,-5.745933,-5.766490,-5.875548,-6.429341,-6.464852,-7.066983,-7.994920,-8.302325,-8.638722,-9.612672,-9.879814,-9.891404,-10.897042,-11.370221,-12.029855,-12.417732,-14.337742,-16.133043


In [60]:
grid.best_estimator_.named_steps['union'].transformer_list[0][1].named_steps['vect']

TfidfVectorizer(analyzer=<method 'split' of 'str' objects>, binary=False,
        decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=0.6,
        max_features=None, min_df=1, ngram_range=(1, 2), norm='l2',
        preprocessor=None, smooth_idf=True, stop_words='english',
        strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [65]:
grid.best_estimator_.named_steps['union'].transformer_list[1][1].named_steps['dict'].get_feature_names()

['ADJ',
 'ADP',
 'ADV',
 'AUX',
 'CCONJ',
 'DET',
 'INTJ',
 'NOUN',
 'NUM',
 'PART',
 'PRON',
 'PROPN',
 'SCONJ',
 'SYM',
 'VERB',
 'length']

In [ ]:
cv_results = pd.DataFrame()
cv_results['mean_test_score'] = (grid.cv_results_['mean_test_score'] * 100).round(2)
cv_results['std_test_score'] = (grid.cv_results_['std_test_score'] * 100).round(2)
cv_results['mean_train_score'] = (grid.cv_results_['mean_train_score'] * 100).round(2)
cv_results['std_train_score'] = (grid.cv_results_['std_train_score'] * 100).round(2)
cv_results['mean_fit_time'] = grid.cv_results_['mean_fit_time'].round(2)
cv_results = cv_results.join(pd.DataFrame(grid.cv_results_).filter(like='param_'))
cv_results['split0_test_score'] = (grid.cv_results_['split0_test_score'] * 100).round(2)
cv_results['split1_test_score'] = (grid.cv_results_['split1_test_score'] * 100).round(2)
cv_results['split2_test_score'] = (grid.cv_results_['split2_test_score'] * 100).round(2)
cv_results['split0_train_score'] = (grid.cv_results_['split0_train_score'] * 100).round(2)
cv_results['split1_train_score'] = (grid.cv_results_['split1_train_score'] * 100).round(2)
cv_results['split2_train_score'] = (grid.cv_results_['split2_train_score'] * 100).round(2)
cv_results.sort_values('std_test_score', ascending=True, axis=0)